<a href="https://colab.research.google.com/github/sainivas-99/License-Plate-Detection/blob/main/LPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics -qq
!pip install easyocr -qq
!pip install GPUtil -qq
!pip install pytesseract -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.9/722.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [10]:
import os
import shutil
import glob
import yaml
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2

import ultralytics
from ultralytics import YOLO

import easyocr

import xml.etree.ElementTree as xet
from bs4 import BeautifulSoup

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import pytesseract

In [11]:
def make_folder(path):
    if not os.path.exists(path):
        os.mkdir(path)

def read_xml(file):
    labels = ['filename', 'width', 'height', 'xmin', 'ymin', 'xmax', 'ymax']
    data = BeautifulSoup(file,"xml")
    d = dict()

    for label in labels:
        txt = data.find(label).text
        if all(c.isdigit() for c in txt):
            d[label] = int(txt)
        else:
            d[label] = txt
    return d

def xml_to_txt(path, height, width):

    with open(path, 'r') as f:
        txt = read_xml(f.read())

    xmin = int(txt['xmin'])
    ymin = int(txt['ymin'])
    xmax = int(txt['xmax'])
    ymax = int(txt['ymax'])

    x_center = (xmin+(xmax-xmin)/2)/width
    y_center = (ymin+(ymax-ymin)/2)/height

    w = (xmax-xmin)/width
    h = (ymax-ymin)/height

    return f'0 {x_center} {y_center} {w} {h}'

In [12]:
total_images = glob.glob('/content/drive/MyDrive/Colab Notebooks/ML/datasets/images/*.png')

In [13]:
indices = np.random.choice(len(total_images), size=int(0.2*len(total_images)), replace= False )
val_indices = np.random.choice(indices, size=int(0.2*len(indices)), replace= False)

In [14]:
make_folder('/content/drive/MyDrive/Colab Notebooks/ML/datasets/output')

In [15]:
sets = ['train','valid','test']
dirs = ['images','labels']
for set in sets:
    path1 = '/content/drive/MyDrive/Colab Notebooks/ML/datasets/output'+f'/{set}'
    make_folder(path1)
    for dir in dirs:
        path2 = path1+f'/{dir}'
        make_folder(path2)
        print(path2)

/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/train/images
/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/train/labels
/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/valid/images
/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/valid/labels
/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/test/images
/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/test/labels


In [16]:
for i, img_path in enumerate(total_images):
    if i in indices:
        dataset = 'test'
    elif i in val_indices:
        dataset = 'valid'
    else:
        dataset = 'train'
    dataset_type_path = '/content/drive/MyDrive/Colab Notebooks/ML/datasets/output'+f'/{dataset}'

    image = cv2.imread(img_path)
    img_height, img_width,_ = image.shape

    annotations_path = img_path.replace('images','annotations').replace('png','xml')
    boxes_info = xml_to_txt(annotations_path, img_height, img_width)

    img_file_name = img_path.split('/')[-1]
    txt_file_name = img_file_name.replace('.png','.txt')
    new_img_path = dataset_type_path+'/images/'+img_file_name
    new_label_path = dataset_type_path+'/labels/'+txt_file_name
    shutil.copy2(img_path, new_img_path)

    txt = open(new_label_path,"w")
    txt.write(boxes_info)
    txt.close()


In [17]:
yaml_details =  {
                        'names' : 'vehicle_plate',
                        'nc' : 1,
                        'train' : '/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/train',
                        'val' : '/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/valid',
                        'test' : '/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/test'
                }

with open('/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/plate.yaml','w+') as file:
    yaml.safe_dump(yaml_details, file)

with open('/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/plate.yaml','r') as file:
    plate_yaml = yaml.safe_load(file)


In [18]:
license_plate_yolo = YOLO('yolov8s.pt')

100%|██████████| 21.5M/21.5M [00:00<00:00, 190MB/s]
